# Part 1: BST Traversal

## Problem 1

In [98]:
from enum import Enum
from TreeTraversal import BinaryTree, DFSTraversal

class DFSTraversalTypes(Enum):
    PREORDER = 1
    INORDER = 2
    POSTORDER = 3
 

### Demo

In [109]:
# fill in tree
input_array = [3, 9, 2, 11]
bt = BinaryTree() 
for val in input_array:
    bt.insert(val)

# inorder
print('inorder')
traversal = DFSTraversal(bt, DFSTraversalTypes.INORDER) 
for val in traversal: 
    print(val)
        
# preorder
print('\npreorder')
traversal = DFSTraversal(bt, DFSTraversalTypes.PREORDER)
for val in traversal: 
    print(val)
    
# postorder
print('\npostorder')
traversal = DFSTraversal(bt, DFSTraversalTypes.POSTORDER) 
for val in traversal: 
    print(val)
    
# back to inorder
print('\nswitch from postorder back to inorder')
traversal = DFSTraversal(bt, DFSTraversalTypes.POSTORDER) 
traversal.changeTraversalType(DFSTraversalTypes.INORDER)
for val in traversal: 
    print(val)

inorder
2
3
9
11

preorder
3
2
9
11

postorder
2
11
9
3

switch from postorder back to inorder
2
3
9
11


# Part 2: Iterators

In [270]:
import numpy as np
import random

class Markov:
    def __init__(self, first_day):
        self.data = None
        self.previous_day = first_day
        self.weather = {
            'sunny': 0,
            'cloudy': 1,
            'rainy': 2,
            'snowy': 3,
            'windy': 4,
            'hailing': 5
        }

    def load_data(self, array):
        self.data = array
        self.idx = 0

    def get_prob(self, previous_day, following_day):
        before_idx = self.weather[previous_day]
        after_idx = self.weather[following_day]

        try:
            prob = self.data[before_idx][after_idx]
            return prob
        except TypeError: # catch if no data file loaded
            error_msg = 'please load data file first with load_data method'
            return error_msg

    def __iter__(self):
        return self

    def __next__(self): 
        try:
            next_day_prob = []
            for next_day in self.weather:
                print(next_day)
                next_day_prob.append(self.get_prob(self.previous_day, next_day))
            result = next_day_prob[self.idx]
        except IndentationError:
            raise StopIteration
        self.idx += 1

In [272]:
x = Markov('sunny')
weather = np.genfromtxt('weather.csv', delimiter=',')
x.load_data(weather)